In [ ]:
    questions_dir = "D:/mgr/nlp/core/data_processing_scripts/q_a/questions"
    answers_dir = ":/mgr/nlp/core/data_processing_scripts/q_a/answers"

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from tqdm import tqdm
import json
from pathlib import Path

def load_model_and_tokenizer(model_name="facebook/opt-1.3b"):
    """Load model and tokenizer with appropriate settings for inference."""
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available. Please check your PyTorch installation and GPU drivers.")
    
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="cuda:0"
    )
    return model, tokenizer

def load_qa_pairs(questions_dir, answers_dir):
    """Load all question-answer pairs from the directories."""
    qa_pairs = []
    
    # Get all question files
    question_files = sorted(Path(questions_dir).glob("*.txt"))
    print(f"Found {len(question_files)} question files.")
    
    for q_file in question_files:
        # Get corresponding answer file
        a_file = Path(answers_dir) / q_file.name

        print(f"Processing {question_files}...")
        
        if a_file.exists():
            print(f"Processing {a_file.name}...")
            # Read all questions and answers
            with open(q_file, 'r', encoding='utf-8') as qf:
                questions = [q.strip() for q in qf.readlines() if q.strip()]
            with open(a_file, 'r', encoding='utf-8') as af:
                answers = [a.strip() for a in af.readlines() if a.strip()]
            
            # Check if the number of questions and answers match
            if len(questions) != len(answers):
                print(f"Warning: Mismatch in number of questions ({len(questions)}) and answers ({len(answers)}) in {q_file.name}")
                continue
            
            # Pair questions and answers
            for i, (question, answer) in enumerate(zip(questions, answers)):
                qa_pairs.append({
                    'file': q_file.name,
                    'line_number': i + 1,
                    'question': question,
                    'reference_answer': answer
                })
    
    return qa_pairs

def generate_answer(model, tokenizer, question, max_length=512):
    """Generate an answer for a given question."""
    prompt = f"Question: {question}\nAnswer:"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.cuda.amp.autocast():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("Answer:", 1)[1].strip()
    return answer

def main():
    # Configuration
    questions_dir = "D:/mgr/nlp/core/data_processing_scripts/q_a/questions"
    answers_dir = "D:/mgr/nlp/core/data_processing_scripts/q_a/answers"
    output_file = "evaluation_results.json"
    model_name = "facebook/opt-1.3b"
    
    # Load model and tokenizer
    print(f"Loading model: {model_name}")
    model, tokenizer = load_model_and_tokenizer(model_name)
    
    # Load QA pairs
    print("Loading QA pairs...")
    qa_pairs = load_qa_pairs(questions_dir, answers_dir)
    
    # Generate answers
    results = []
    print("Generating answers...")
    for qa in tqdm(qa_pairs):
        try:
            generated_answer = generate_answer(model, tokenizer, qa['question'])
            result = {
                'file': qa['file'],
                'line_number': qa['line_number'],
                'question': qa['question'],
                'generated_answer': generated_answer,
                'reference_answer': qa['reference_answer']
            }
            results.append(result)
        except Exception as e:
            print(f"Error processing {qa['file']} line {qa['line_number']}: {str(e)}")
    
    # Save results
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"Evaluation completed. Results saved to {output_file}")

if __name__ == "__main__":
    main()

Loading model: facebook/opt-1.3b
Using GPU: NVIDIA GeForce RTX 4070 SUPER
Loading QA pairs...
Found 11 question files.
Processing [WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Balladyna_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Dziady_(Mickiewicz)_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Konrad_Wallenrod_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Kordian_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Lalka_(Prus)_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Ogniem_i_mieczem_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Pan_Tadeusz_(wyd._1834)_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Pan_Wołodyjowski_questions.txt'), WindowsPath('D:/mgr/nlp/core/data_processing_scripts/q_a/questions/Poto

0it [00:00, ?it/s]

Evaluation completed. Results saved to evaluation_results.json
